# util

## logger.py

In [90]:
from io import StringIO
from time import localtime, mktime, gmtime
import logging, sys


class Logger(logging.Logger):
    _runtime = None
    _buffer = None

    @classmethod
    def kst(cls, *args):
        return localtime(mktime(gmtime()) + 9 * 3600)

    def __init__(self, name: str):
        super().__init__(name=name, level=logging.INFO)
        self.propagate = False
        formatter = logging.Formatter(
            fmt=f"%(asctime)s %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S"
        )
        formatter.converter = self.kst

        stream = logging.StreamHandler(stream=sys.stdout)
        stream.setLevel(logging.INFO)
        stream.setFormatter(formatter)
        self.addHandler(stream)

        self._buffer = StringIO()
        memory = logging.StreamHandler(stream=self._buffer)
        memory.setLevel(logging.INFO)
        memory.setFormatter(formatter)
        self.addHandler(memory)
        return

    def __call__(self, msg: str):
        self.info(msg=msg)
        return

    @property
    def stream(self) -> str:
        return self._buffer.getvalue()

    def clear(self):
        self._buffer.truncate(0)
        self._buffer.seek(0)
        return

## mailing.py

In [91]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP


class Mail(MIMEMultipart):
    """
    e-mail 전송 서비스
    """

    def __init__(self):
        super().__init__()
        self['From'] = 'snob.labwons@gmail.com'
        self.content = ""
        return

    @property
    def Subject(self) -> str:
        return self['Subject']

    @Subject.setter
    def Subject(self, _subject: str):
        self['Subject'] = _subject

    @property
    def To(self) -> str:
        return self['To']

    @To.setter
    def To(self, _to: str):
        self['To'] = _to

    def send(self, *args):
        self.attach(MIMEText(self.content, *args))
        with SMTP('smtp.gmail.com', 587) as server:
            server.ehlo()
            server.starttls()
            server.login(self['From'], "puiz yxql tnoe ivaa")
            server.send_message(self)
        return

# crypto

## bithumb

### ticker.py

In [ ]:
from pandas import DataFrame, Series
from typing import Union


class Ticker:

    url:str = "https://api.bithumb.com/v1"
    headers = {"accept": "application/json"}
    rename = {
        "candle_date_time_kst": "datetime",
        "opening_price": "open",
        "high_price": "high",
        "low_price": "low",
        "trade_price": "close",
        "candle_acc_trade_price": "amount",
        "candle_acc_trade_volume": "volume",
        "trade_volume": 'volume',
        "ask_bid": "quote",
        "datetime": "datetime"
    }

    def __init__(self, ticker: str):
        self.ticker = ticker
        return

    def __repr__(self):
        return repr(self.snapShot())

    @classmethod
    def _fetch_(cls, url:str, **kwargs) -> Union[DataFrame, Series]:
        """
        url: https://api.bithumb.com 의 api 데이터 취득
        base url의 하위 주소를 입력하여 응답을 pandas Series 또는 DataFrame으로 변환

        Args:
            url     (str)   : 주소, base url이 제외된 하위 주소
            kwargs  (dict)  : Series 또는 DataFrame 변환 시 전달할 Keywoard Arguments

        Returns:
            Union[Series, DataFrame] :
        """
        resp = requests \
               .get(f"{cls.url}{url}", headers=cls.headers) \
               .json()
        return Series(resp[0], **kwargs) if len(resp) == 1 else DataFrame(resp, **kwargs)

    def snapShot(self) -> Series:
        """
        market                               KRW-BTC
        trade_date                          20251128
        trade_time                            021901
        trade_date_kst                      20251128
        trade_time_kst                        111901
        trade_timestamp                1764328741157
        opening_price                      135796000
        high_price                         137125000
        low_price                          135372000
        trade_price                        135442000
        prev_closing_price                 135796000
        change                                  FALL
        change_price                          354000
        change_rate                           0.0026
        signed_change_price                  -354000
        signed_change_rate                   -0.0026
        trade_volume                        0.000073
        acc_trade_price           29804029073.873871
        acc_trade_price_24h      111366853330.307678
        acc_trade_volume                  218.675501
        acc_trade_volume_24h              815.641639
        highest_52_week_price              179734000
        highest_52_week_date              2025-10-10
        lowest_52_week_price               110000000
        lowest_52_week_date               2024-12-04
        timestamp                      1764328741157
        Name: KRW-BTC, dtype: object
        """
        return self._fetch_(f"/ticker?markets={self.ticker}", name=self.ticker)

    def ohlcv(self, period: str = 'd', *args, **kwargs) -> DataFrame:
        """
                                 open	     high	      low	    close	      amount	     volume
        datetime
        2025-12-08T14:30:00	135976000	136192000	135976000	136032000	3.107575e+08	 2283670.13
        2025-12-08T14:00:00	135976000	136167000	135544000	135975000	1.790029e+09	13176614.54
        2025-12-08T13:30:00	135948000	136185000	135811000	135977000	1.024618e+09	 7535912.03
        2025-12-08T13:00:00	135782000	136038000	135700000	135948000	7.361658e+08	 5419830.47
        2025-12-08T12:30:00	136113000	136113000	135702000	135761000	1.072309e+09	 7893673.71
        ...	...	...	...	...	...	...
        2025-12-04T13:00:00	139211000	139232000	138978000	139048000	1.259391e+09	 9056382.13
        2025-12-04T12:30:00	139232000	139250000	138999000	139199000	1.363260e+09	 9800596.39
        2025-12-04T12:00:00	139559000	139650000	139160000	139244000	1.809427e+09	12977284.73
        2025-12-04T11:30:00	139374000	139618000	139148000	139559000	2.374796e+09	17033654.61
        2025-12-04T11:00:00	138590000	139590000	138500000	139374000	4.618125e+09	33227210.66
        200 rows × 6 columns

        :param period:
        :param args:
        :param kwargs:
        :return:
        """
        period = {'d': 'days', 'min': 'minutes', 'w': 'weeks', 'm': 'months'}[period.lower()]
        if period == 'minutes':
            unit = args[0] if args else kwargs.get("unit", 60)
            query = f'/candles/{period}/{unit}?market={self.ticker}&count={kwargs.get('count', 200)}'
        else:
            query = f'/candles/{period}?market={self.ticker}&count={kwargs.get('count', 200)}'

        data = self._fetch_(query)
        cols = {k: v for k, v in self.rename.items() if k in data.columns}
        data = data.rename(columns=cols)[cols.values()]
        data = data.set_index(keys='datetime')
        data['volume'] = data['volume'] * 1e+6
        return data

    def execution(self, count: int = 100) -> DataFrame:
        """
        체결 내역

                                close	  volume  quote
        datetime
        2025-12-08 14:42:39	136108000	0.005988	BID
        2025-12-08 14:42:39	136100000	0.003500	BID
        2025-12-08 14:42:39	136099000	0.000087	BID
        2025-12-08 14:42:39	136082000	0.003211	BID
        2025-12-08 14:42:21	136082000	0.000050	BID
        ...	...	...	...
        2025-12-08 14:39:29	136101000	0.000073	ASK
        2025-12-08 14:39:29	136107000	0.001832	BID
        2025-12-08 14:39:18	136107000	0.003049	BID
        2025-12-08 14:39:10	136107000	0.000004	ASK
        2025-12-08 14:39:07	136108000	0.000062	ASK
        100 rows × 3 columns

        :param count:
        :return:
        """
        data = self._fetch_(f'/trades/ticks?market={self.ticker}&count={min(count, 500)}')
        data['datetime'] = pd.to_datetime(data['trade_date_utc'].astype(str) + ' ' + data['trade_time_utc'].astype(str))
        data['datetime'] = data['datetime'] + pd.Timedelta(hours=9)
        cols = {k: v for k, v in self.rename.items() if k in data.columns}

        data = data.rename(columns=cols)[cols.values()]
        data = data.set_index(keys='datetime')
        return data

    def order(self) -> DataFrame:
        """
        주문 내역

                            ask_price	bid_price  ask_size  bid_size
        datetime
        2025-12-08 14:43:42	136107000	136082000	 0.0012    0.0007
        2025-12-08 14:43:42	136108000	136059000	 0.0356	   0.0735
        2025-12-08 14:43:42	136109000	136026000	 0.1031	   0.0043
        2025-12-08 14:43:42	136112000	136025000	 0.0328	   0.0346
        2025-12-08 14:43:42	136117000	136014000	 0.0036	   0.0029
        ... ... ... ... ...
        2025-12-08 14:43:42	136182000	135977000	 0.0038	   0.0588
        2025-12-08 14:43:42	136185000	135976000	 0.0093	   0.0907
        2025-12-08 14:43:42	136186000	135975000	 0.0322	   0.1778
        2025-12-08 14:43:42	136188000	135974000	 0.0007	   0.0000
        2025-12-08 14:43:42	136192000	135973000	 0.0010	   0.0722
        :return:
        """
        base = self._fetch_(f'/orderbook?markets={self.ticker}')
        data = DataFrame(base['orderbook_units'])
        data['datetime'] = pd.to_datetime(base['timestamp'], unit='ms') + pd.Timedelta(hours=9)
        data['datetime'] = data['datetime'].dt.strftime("%Y-%m-%d %H:%M:%S")
        return data.set_index(keys='datetime')


### market.py

In [ ]:
from collections import deque
from pandas import DataFrame, Series
from typing import Union
from logging import Logger as Log
import requests


class Market:
    url: str = "https://api.bithumb.com/v1"
    headers = {"accept": "application/json"}

    rename = {
        'market': 'ticker',
        'english_name': 'name',
        # 'market_warning': 'warning',
        # 'korean_name': 'kor',
        'warning_type': 'warning',
        'end_date': 'warning_end'
    }

    def __init__(self, logger: Union[Log, None] = None):
        self._mem_ = deque(maxlen=5)
        self._log_ = logger
        return

    def __iter__(self):
        for ticker in self.tickers.index:
            yield ticker

    def _repr_html_(self):
        return self.tickers._repr_html_()

    @classmethod
    def _fetch_(cls, url: str, **kwargs) -> Union[DataFrame, Series]:
        """
        url: https://api.bithumb.com 의 api 데이터 취득
        base url의 하위 주소를 입력하여 응답을 pandas Series 또는 DataFrame으로 변환

        Args:
            url     (str)   : 주소, base url이 제외된 하위 주소
            kwargs  (dict)  : Series 또는 DataFrame 변환 시 전달할 Keywoard Arguments

        Returns:
            Union[Series, DataFrame] :
        """
        resp = requests \
            .get(f"{cls.url}{url}", headers=cls.headers) \
            .json()
        return Series(resp[0], **kwargs) if len(resp) == 1 else DataFrame(resp, **kwargs)

    @classmethod
    def _fetch_tickers(cls) -> DataFrame:
        data = cls._fetch_('/market/all?isDetails=true')
        cols = {k: v for k, v in cls.rename.items() if k in data.columns}
        data = data.rename(columns=cols)[cols.values()]
        data = data[data['ticker'].str.startswith('KRW')]
        # data["warning"] = data["warning"].replace("NONE", None)
        return data.set_index(keys='ticker')

    @classmethod
    def _fetch_warnings(cls) -> DataFrame:
        data = cls._fetch_('/market/virtual_asset_warning')
        cols = {k: v for k, v in cls.rename.items() if k in data.columns}
        data = data.rename(columns=cols)[cols.values()]
        data = data[data['ticker'].str.startswith('KRW')]
        return data.set_index(keys='ticker')

    @property
    def baseline(self) -> DataFrame:
        if not self._mem_:
            self.update_baseline(period='min', unit=60)
        return list(self._mem_)[-1]

    @property
    def tickers(self) -> DataFrame:
        """
                            name	                          warning	        warning_end
        ticker
        KRW-BTC	         Bitcoin	                              NaN	                NaN
        KRW-ETH	        Ethereum	                              NaN	                NaN
        KRW-ETC	Ethereum Classic	                              NaN	                NaN
        KRW-XRP	             XRP	                              NaN	                NaN
        KRW-BCH	    Bitcoin Cash	                              NaN	                NaN
        ...	                 ...	                              ...	                ...
        KRW-MMT	        Momentum	                              NaN	                NaN
        KRW-MET	         Meteora	                              NaN	                NaN
        KRW-KITE	        Kite	                              NaN	                NaN
        KRW-TRUST	   Intuition	DEPOSIT_AMOUNT_SUDDEN_FLUCTUATION	2025-12-10 07:04:59
        KRW-PIEVERSE	Pieverse	                              NaN	                NaN
        450 rows × 3 columns
        """
        return self._fetch_tickers().join(self._fetch_warnings())

    def update_baseline(self, period: str = 'd', *args, **kwargs):
        if self._log_: self._log_.info('UPDATE OHLCV BASELINE')
        objs = {}
        for ticker in self.tickers.index:
            try:
                objs[ticker] = Ticker(ticker).ohlcv(period=period, *args, **kwargs)
            except KeyError:
                if self._log_: self._log_.info(f'>>> FAILED TO UPDATE OHLCV: {ticker}')
                continue
        self._mem_.append(pd.concat(objs, axis=1).sort_index(ascending=True).tail(200))
        if self._log_: self._log_.info('UPDATE OHLCV BASELINE SUCCEESS')
        return



# analysis

## indicator.py

In [94]:
from pandas import DataFrame


class Indicator:

    def __init__(self, baseline: DataFrame):
        self.data = baseline.copy()
        return

    def __call__(self, *tickers) -> DataFrame:
        idx = pd.IndexSlice
        selector = list(tickers)
        return self.data.loc[:, idx[selector, :]]

    def __iter__(self):
        for col in self.data.columns.get_level_values(0).unique():
            yield col

    def __contains__(self, col: str):
        return col in self.data['KRW-BTC'].columns

    def __getitem__(self, col: str):
        return self.data.xs(col, axis=1, level=1)

    def __setitem__(self, col: str, series):
        series.columns = pd.MultiIndex.from_product([series.columns, [col]])
        self.data = pd.concat([self.data, series], axis=1).sort_index(axis=1)
        return

    def __delitem__(self, col: str):
        if not col in self:
            return
        mask = self.data.columns.get_level_values(1) == col
        self.data = self.data.drop(columns=self.data.columns[mask])
        return

    def _repr_html_(self):
        return self.data._repr_html_()

    def _set_columns(self, **kwargs):
        for column, series in kwargs.items():
            self[column] = series
        return

    def _del_columns(self, *cols):
        for col in cols:
            del self[col]
        return

    def install(self):
        self.add_tp()
        self.add_bb()
        return

    def add_tp(self):
        self['tp'] = round((self['high'] + self['low'] + self['close']) / 3, 4)
        return

    def add_bb(self, basis: str = 'tp', window: int = 20, std: int = 2):
        basis = basis if basis in self else 'close'
        dev = self[basis].rolling(window).std()
        mid = self[basis].rolling(window).mean()
        up = mid + std * dev
        dn = mid - std * dev
        self._set_columns(
            mid=mid,
            bb_upper=up,
            bb_lower=dn,
            tr_upper=mid + (std / 2) * dev,
            tr_lower=mid - (std / 2) * dev,
            bb_width=((up - dn) / mid) * 100
        )
        return

# indicator = Indicator(coins.baseline)
# indicator.install()
# indicator.add_tp()
# indicator.add_bb()
# indicator

# indicator['amount']

## strategy.py

In [95]:
if not "Indicator" in globals():
    from src.analysis.indicator import Indicator
from pandas import DataFrame


class Strategy(Indicator):

    @staticmethod
    def to_report(sig: DataFrame) -> DataFrame:
        lap = pd.to_datetime(datetime.now(ZoneInfo('Asia/Seoul'))).tz_localize(None)
        sig = sig[pd.to_datetime(sig.index) >= (lap - pd.Timedelta(hours=48))]
        sig.columns = [c.replace("KRW-", "") for c in sig.columns]
        return sig.apply(lambda row: ','.join(sig.columns[row.notna()]), axis=1) \
            .replace("", None) \
            .dropna()

    def squeeze_expand(
        self,
        width_mode: str = 'min',
        width_window: int = 7,
        volume_window: int = 7
    ):
        # 밴드 폭 전략
        # @width_mode == 'min':
        #     @width_window에 대해 최소 값인 경우
        #     * 단기 가격 변동성을 고려할 때 사용
        # @width_mode == 'lower':
        #     @width_rank 보다 작은 경우
        #     * 중자아기 가격 변동성을 고려할 때 사용
        if width_mode == 'min':
            self['sig_squeeze_expand'] = (
                (self['bb_width'] == self['bb_width'].rolling(width_window).min()) &
                (self['close'] >= self['bb_upper']) &
                ((self['close'] - self['open']) >= (self['bb_upper'] - self['mid'])) &
                (self['volume'] >= self['volume'].rolling(volume_window).mean())
            ).astype(int).replace(0, None)
        elif width_mode == 'lower':
            pass
        else:
            raise KeyError
        return self['sig_squeeze_expand']

# coins = Coins()
# strategy = Strategy(coins.baseline)
# strategy.install()
# strategy.squeeze_expand()
# strategy.to_report(strategy['sig_squeeze_expand'])



# bot

## bithumb.py

In [106]:
from datetime import datetime
from zoneinfo import ZoneInfo
import pandas as pd


TZ = ZoneInfo('Asia/Seoul')
TIMEUNIT = 'min'
INTERVAL = 30

logger = Logger('BOT@v1')
market = Market(logger=logger)
market.update_baseline(period=TIMEUNIT, unit=INTERVAL, count=200)

strategy = Strategy(market.baseline)
strategy.install()
signal = strategy.squeeze_expand()
report = strategy.to_report(signal)

diff = datetime.now(TZ) - pd.to_datetime(report.index[-1]).tz_localize(TZ)
if TIMEUNIT != 'min' or (TIMEUNIT == 'min' and int(diff.total_seconds() / 60) <= INTERVAL):

    clock = datetime.now(TZ).strftime("%Y/%m/%d %H:%M")
    logger(f'□ 감지 시간: {report.index[-1].replace("-", "/").replace("T", " ")[:-3]}')
    logger(f'□ 현재 시간: {clock}')
    for ticker in report.values[-1].split(","):
        coin = Ticker(ticker=f'KRW-{ticker}')
        snap = coin.snapShot()
        logger(f'□ TICKER: {ticker}')
        logger(f'  - LINK: https://m.bithumb.com/react/trade/chart/{ticker}-KRW')
        logger(f'  - 현재가: {snap["trade_price"]}원')
        logger(f'  - 등락률: {100 * snap["signed_change_rate"]:.2f}%')
        logger(f'  - 거래대금: {snap["acc_trade_price_24h"] / 1e+8:.2f}억원')

    text = []
    for line in str(logger.stream).splitlines():
        line = line[20:]
        if "TICKER" in line:
            ticker = line[line.find(":") + 2:]
            url = f'https://m.bithumb.com/react/trade/chart/{ticker}-KRW'
            line = line.replace(": ", f': <a href="{url}">') + "</a>"
        if "LINK" in line:
            continue
        text.append(line)

    stream = f"""<!doctype html><html><body><p>{"<br>".join(text)}</p>{report.to_html()}</body></html>"""
    mail = Mail()
    mail.Subject = f'TRADER@v1 ON {clock}'
    mail.To = 'jhlee_0319@naver.com'
    mail.content = stream
    mail.send("html", "utf-8")

else:
    logger('NO SIGNALS DETECTED, SYSTEM ABORT')




2025-12-09 11:30:25 UPDATE OHLCV BASELINE
2025-12-09 11:32:50 UPDATE OHLCV BASELINE SUCCEESS
2025-12-09 11:32:50 NO SIGNALS DETECTED, SYSTEM ABORT
